# More power method (continuation of lecture 2)

In this continuation of lecture 2, we will see that having a good abstraction of hardware resources allows us to run the **same code** in parallel.

"Parallel computing will have made it when we never have to know any of the internal details." Alan Edelman

## Using parallel hardware

In [2]:
function power_method(M, v)
    T = eltype(v)
    for i in 1:100
        v = M*v        # repeatedly creates a new vector and destroys the old v
        v /= T(norm(v))
    end
    
    return v, T(norm(M*v)) / T(norm(v))  # or  (M*v) ./ v
end

power_method (generic function with 1 method)

### ArrayFire for calculations on the GPU

ArrayFire is a Julia package that wraps the [`arrayfire`](http://arrayfire.com/) library for doing efficient array calculations on the GPU (graphics card). 

The [ArrayFire.jl](https://github.com/JuliaComputing/ArrayFire.jl) package provides a Julia wrapper for the `arrayfire` library. Note that **you must separately install the `arrayfire` library on your computer, before using this package**.

In [2]:
# First install the arrayfire library from the arrayfire homepage.
# On Mac you can use [Homebrew](http://brew.sh/), which you must first install and then execute
# brew install arrayfire

In [3]:
Pkg.add("ArrayFire")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of ArrayFire
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [6]:
using ArrayFire

`ArrayFire.jl` provides an easy way to create and manipulate arrays on the GPU. 
It is easy (although may be expensive!) to pass arrays backwards and forwards from the CPU to the GPU:

First we create a standard Julia matrix (on the CPU):

In [6]:
M = [2 1; 1 1.]

2×2 Array{Float64,2}:
 2.0  1.0
 1.0  1.0

We can copy this into an array on the GPU with 

In [7]:
MM = AFArray(M);   # can't print out these objects unless use master branch of ArrayFire

2×2 ArrayFire.AFArray{Float64,2}:
 2.0  1.0
 1.0  1.0

[Note that in order to print out these objects on Julia v0.5, you require (on 15 September 2016) the master (i.e. development) branch of the package, which may be obtained by executing

    Pkg.checkout("ArrayFire")
]

This calls a **constructor** of the `AFArray` object, that constructs an array on the GPU by copying the data provided.

We do the same for a vector:

In [10]:
v = [1., 1]
vv = AFArray(v)

2-element ArrayFire.AFArray{Float64,1}:
 1.0
 1.0

and then multiply the matrix and vector **on the GPU**:

In [12]:
MM * vv

2-element ArrayFire.AFArray{Float64,2}:
 3.0
 2.0

We see that the `*` operation indeed has a method defined to perform the matrix-vector multiplication and create the result as a new object (in fact, a $2 \times 1$ matrix) on the GPU.

We are thus now able to call `power_method` directly:

In [14]:
vec, val = power_method(MM, vv)

([0.850651,0.525731],2.618033988749895)

In [15]:
vec

2-element ArrayFire.AFArray{Float64,1}:
 0.850651
 0.525731

We see that the result of the calculation is indeed still an object on the GPU.

**Exercise**: Compare the time on GPU and CPU. Since execution on the GPU is asynchronous, it is necessary to synchronise:

In [7]:
function runGPU(MM, vv)
    power_method(MM, vv)
    sync()   # wait for the GPU to finish
end

runGPU (generic function with 1 method)

In [8]:
@time runGPU(MM, vv)

LoadError: LoadError: UndefVarError: MM not defined
while loading In[8], in expression starting on line 184

In [21]:
@time power_method(M, v)

  0.000026 seconds (307 allocations: 20.609 KB)


([0.850651,0.525731],2.618033988749895)

In [12]:

n = 10000
M = rand(Float32, n, n)  # GPUs are much more efficient with Float32s
M = (M + M')/2
v = rand(Float32, n, 1);

In [17]:
typeof(MM),typeof(vv)

(ArrayFire.AFArray{Float32,2},ArrayFire.AFArray{Float32,2})

In [13]:
@time power_method(M, v)

  8.195866 seconds (508 allocations: 7.686 MB)


(
Float32[0.00996414; 0.00998084; … ; 0.0100507; 0.0100459],

4999.5728f0)

In [14]:
MM = AFArray(M)
vv = AFArray(v)

#@time power_method(M, v);
@time runGPU(MM, vv);


  0.803818 seconds (3.02 k allocations: 62.891 KB)


In [23]:
typeof(norm(MM*vv))

Float64

On my machine, the CPU version is much faster for small arrays, while the GPU version is 3 times faster for matrices of linear size $n=10000$.

### DistributedArrays for large arrays spread across different processors

The Julia package [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl) defines a `DArray` ("distributed array") type, which provides an abstraction that looks like a standard Julia array, but is spread across several different processors.

Since modern desktops and laptops often have multiple cores, we can use this.

First we allow Julia access to more processes:

In [36]:
addprocs(4)   # add cores to your Julia process

4-element Array{Int64,1}:
 2
 3
 4
 5

In [37]:
using DistributedArrays

There are several ways to create `DArray`s:

In [38]:
M = drand(10, 10)

10×10 DistributedArrays.DArray{Float64,2,Array{Float64,2}}:
 0.248988  0.153889   0.664526  0.661067   …  0.971092  0.824504   0.830846
 0.641347  0.374573   0.59397   0.557687      0.19684   0.532591   0.159857
 0.803745  0.0318901  0.61925   0.0707356     0.893829  0.738849   0.83417 
 0.237734  0.0881233  0.405531  0.25496       0.709861  0.0876057  0.115356
 0.651232  0.816379   0.314216  0.490344      0.584675  0.521554   0.382791
 0.706556  0.758815   0.955861  0.0387312  …  0.260452  0.154861   0.609741
 0.838201  0.124591   0.846526  0.840725      0.759394  0.983453   0.338123
 0.435181  0.296503   0.697961  0.763113      0.228731  0.306705   0.807473
 0.203791  0.589002   0.394243  0.69337       0.676374  0.467022   0.841446
 0.382667  0.92903    0.432306  0.331317      0.832022  0.640685   0.948066

If we really need to, we can find out where Julia is storing each piece of the array:

In [39]:
M.indexes

2×2 Array{Tuple{UnitRange{Int64},UnitRange{Int64}},2}:
 (1:5,1:5)   (1:5,6:10) 
 (6:10,1:5)  (6:10,6:10)

This shows that the array was divided up into equal pieces on each of the four processors.

In [43]:
v = drand(10)

10-element DistributedArrays.DArray{Float64,1,Array{Float64,1}}:
 0.0217203
 0.42808  
 0.360108 
 0.210527 
 0.118268 
 0.773549 
 0.863915 
 0.811474 
 0.884174 
 0.0721028

In [44]:
M * v

10-element DistributedArrays.DArray{Float64,1,Array{Float64,1}}:
 3.07848
 2.0511 
 3.03843
 1.58895
 2.80143
 2.38656
 2.85551
 2.28942
 2.67886
 3.0865 

Again, we see that `*` has been defined for these objects, so once again we can just run

In [45]:
power_method(M, v)

([0.362065,0.265624,0.353252,0.207619,0.330826,0.309607,0.334392,0.317482,0.296875,0.352718],5.471962252198712)

## Operators

Consider the following averaging operator that we could call a random walk or averaging operator:

In [47]:
averaging(n) = 0.5 * SymTridiagonal(zeros(n), ones(n-1))

averaging (generic function with 1 method)

In [48]:
averaging(7)

7×7 SymTridiagonal{Float64}:
 0.0  0.5   ⋅    ⋅    ⋅    ⋅    ⋅ 
 0.5  0.0  0.5   ⋅    ⋅    ⋅    ⋅ 
  ⋅   0.5  0.0  0.5   ⋅    ⋅    ⋅ 
  ⋅    ⋅   0.5  0.0  0.5   ⋅    ⋅ 
  ⋅    ⋅    ⋅   0.5  0.0  0.5   ⋅ 
  ⋅    ⋅    ⋅    ⋅   0.5  0.0  0.5
  ⋅    ⋅    ⋅    ⋅    ⋅   0.5  0.0

In [51]:
v = 1:2:13
averaging(7) * v

7-element Array{Float64,1}:
  1.5
  3.0
  5.0
  7.0
  9.0
 11.0
  5.5

In [52]:
power_method(averaging(7), v)

([0.198757,0.339299,0.479841,0.479841,0.479841,0.339299,0.198757],0.9238795325112868)

Although we have saved some memory by using a `SymTridiagonal` structure, we clearly are still storing far more information than we need to, since this is just "0 on the diagonal and 0.5 on the super- and sub-diagonal".

We can define a new type in Julia to reflect this. We realise that we do **not actually need to store any information inside the "matrix"**. In fact, we will rather define a **linear operator**, just as we would really like to do in mathematics:

In [53]:
type AveragingOp
    # contains *no* information
end

We have a "dummy type" that contains no information. It will be interesting because of "what it can do", i.e. the operations that we define that involve objects of this type.

We create an object of this type, called `A`, with

In [55]:
A = AveragingOp()  # default constructor

AveragingOp()

In [56]:
A

AveragingOp()

We will define what it means to multiply an object of this type by a vector. The simplest case would be

In [57]:
import Base.*  # necessary to overload *

function *(A::AveragingOp, v::AbstractVector)
    v  # just the identity operator
end

* (generic function with 161 methods)

which gives an identity operator:

In [60]:
v = [1, 2, 43]
A*v

3-element Array{Int64,1}:
  1
  2
 43

In [61]:
power_method(A, v)

([0.0232244,0.0464489,0.998651],1.0)

We now define the actual averaging operation. It takes a vector and returns a new vector:

In [62]:
function *(A::AveragingOp, v::AbstractVector)
    [ v[1];    # ; concatenates
      [(v[i-1] + v[i+1])/2  for i in 2:length(v)-1];    # array comprehension
      v[end] 
    ]
end

* (generic function with 161 methods)

In [66]:
v = (1:7).^2
@show v
A*v

v = [1,4,9,16,25,36,49]


7-element Array{Float64,1}:
  1.0
  5.0
 10.0
 17.0
 26.0
 37.0
 49.0

Since `*` now works, we can again just reuse our some generic `power_method` implementation:

In [68]:
power_method(A, v)

([0.0146239,0.0584955,0.131615,0.233982,0.365597,0.52646,0.71657],1.0)

You could worry that `*` is not the correct notation. Mathematically, for an operator $\mathcal{L}$ operating on a vector $\mathbf{v}$, we might write $\mathcal{L} \mathbf{v}$, just using juxtaposition. Unfortunately, we are unable to use this notation in Julia.

We could instead use a `⋅` for juxtaposition. Now that we have defined `*`, we can just do

In [70]:
import Base.⋅
A::AveragingOp ⋅ v = A * v

dot (generic function with 16 methods)

In [71]:
A ⋅ v

7-element Array{Float64,1}:
  1.0
  5.0
 10.0
 17.0
 26.0
 37.0
 49.0

We can even define $\mathcal{L}(\mathbf{v})$:

In [72]:
(A::AveragingOp)(v) = A*v

In [73]:
A(v)

7-element Array{Float64,1}:
  1.0
  5.0
 10.0
 17.0
 26.0
 37.0
 49.0

In [27]:
@which norm(vv)

norm(a::ArrayFire.AFArray) at /Users/dpsanders/.julia/v0.5/ArrayFire/src/linalg.jl:142

In [28]:
?ArrayFire.af_norm

No documentation found.

`ArrayFire.af_norm` is a `Function`.

```
# 1 method for generic function "af_norm":
af_norm(out::Ref, _in::ArrayFire.AFArray, _type::Int64, p::Real, q::Real) at /Users/dpsanders/.julia/v0.5/ArrayFire/src/wrap.jl:1758
```
